# Dashboards and Final Report

This notebook builds interactive dashboards and generates a textual summary
report for the **Yelp Rating Drop Early-Warning System** using the **optimized
Random Forest model**.

It uses the following optimized artifacts:

- `data/processed/reviews_features_optimized.csv`
- `models/rating_drop_model_optimized.pkl`
- `models/feature_list_optimized.txt`
- `models/model_summary_optimized.csv`
- `results/llm_complaint_analysis.csv`


In [1]:
import os
import json
import pickle

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go

# Ensure output directories exist
os.makedirs("dashboards", exist_ok=True)
os.makedirs("reports", exist_ok=True)

print("Libraries imported and output directories ensured.")


Libraries imported and output directories ensured.


## 1. Load Optimized Data and Model Artifacts


In [ ]:
# Load optimized feature dataset
features_path = "data/processed/reviews_features_optimized.csv"
df = pd.read_csv(features_path, parse_dates=["date"])
print(f"Loaded optimized features from: {features_path}")
print("Shape:", df.shape)

# Load optimized model
model_path = "models/rating_drop_model_optimized.pkl"
with open(model_path, "rb") as f:
    model = pickle.load(f)
print(f"Loaded optimized model from: {model_path}")

# Load optimized feature list
feature_list_path = "models/feature_list_optimized.txt"
with open(feature_list_path, "r") as f:
    feature_cols = [line.strip() for line in f.readlines() if line.strip()]
print(f"Loaded {len(feature_cols)} features from: {feature_list_path}")

# Intersect with available columns just in case
feature_cols = [c for c in feature_cols if c in df.columns]
print(f"Using {len(feature_cols)} features present in dataset.")

# Load model summary
model_summary_path = "models/model_summary_optimized.csv"
model_summary = pd.read_csv(model_summary_path).iloc[0]
print(f"Loaded model summary from: {model_summary_path}")

# Load LLM complaint analysis if available
llm_path = "results/llm_complaint_analysis.csv"
llm_df = None
if os.path.exists(llm_path):
    llm_df = pd.read_csv(llm_path)
    print(f"Loaded LLM complaint analysis from: {llm_path}")
    if "date" in llm_df.columns:
        llm_df["date"] = pd.to_datetime(llm_df["date"], errors="coerce")
else:
    print(
        "LLM complaint analysis file not found; some dashboard elements will be skipped."
    )


Loaded optimized features from: data/processed/reviews_features_optimized.csv
Shape: (89878, 41)
Loaded optimized model from: models/rating_drop_model_optimized.pkl
Loaded 31 features from: models/feature_list_optimized.txt
Using 31 features present in dataset.
Loaded model summary from: models/model_summary_optimized.csv
Loaded LLM complaint analysis from: results/llm_complaint_analysis.csv


## 2. Compute Risk Scores Using the Optimized Model


In [ ]:
# Build feature matrix and compute risk scores
X = df[feature_cols].copy()
risk_proba = model.predict_proba(X)[:, 1]
df["risk_score"] = risk_proba

# Attach simple risk flag using optimal threshold from Notebook 2
optimal_threshold = float(model_summary.get("optimal_threshold", 0.29))
df["risk_flag"] = (df["risk_score"] >= optimal_threshold).astype(int)

print("Computed risk scores and risk_flag using optimal threshold:", optimal_threshold)

# Basic sanity check
print(
    df[
        ["business_id", "name", "date", "stars_business", "risk_score", "risk_flag"]
    ].head()
)


Computed risk scores and risk_flag using optimal threshold: 0.29
              business_id                                   name  \
0  -0M0b-XhtFagyLmsBtOe8w                         Paris Wine Bar   
1  -0TffRSXXIlBYVbb5AwfTg  IndeBlue Modern Indian Food & Spirits   
2  -0TffRSXXIlBYVbb5AwfTg  IndeBlue Modern Indian Food & Spirits   
3  -0TffRSXXIlBYVbb5AwfTg  IndeBlue Modern Indian Food & Spirits   
4  -0TffRSXXIlBYVbb5AwfTg  IndeBlue Modern Indian Food & Spirits   

                 date  stars_business  risk_score  risk_flag  
0 2015-06-24 22:10:39             3.5    0.426102          1  
1 2013-06-27 13:44:37             4.5    0.223922          0  
2 2013-08-03 13:56:04             4.5    0.232762          0  
3 2013-09-15 17:18:13             4.5    0.187257          0  
4 2013-09-18 02:20:26             4.5    0.204912          0  


## 3. Business-Level Aggregations


In [ ]:
# Aggregate by business
business_group = df.groupby(["business_id", "name"], as_index=False).agg(
    avg_rating=("stars_business", "mean"),
    latest_rating=("stars_business", "last"),
    avg_risk_score=("risk_score", "mean"),
    max_risk_score=("risk_score", "max"),
    n_reviews=("review_id", "count"),
)

# Sort by average risk score (high to low)
business_group = business_group.sort_values("avg_risk_score", ascending=False)

print("Business-level summary shape:", business_group.shape)
print(business_group.head())


Business-level summary shape: (4094, 7)
                 business_id                         name  avg_rating  \
1755  QX6qD3JEopZSH2SQsmObxg                Xi'an Cuisine         3.0   
3136  kr6FpW5rMNp_OX8Ha4j0Fw  Amasi Restaurant and Hookah         3.5   
2824  gU39t80dXx9LkIZHp6cyCQ                Magic Noodles         3.5   
366   4zEqYybRD1FQssLGaJnNZA                 The Blockley         3.5   
2455  b1pdnbYvBdtTxJOOom5WmA               Andy's Chicken         4.0   

      latest_rating  avg_risk_score  max_risk_score  n_reviews  
1755            3.0        0.616899        0.645203          6  
3136            3.5        0.602955        0.602955          1  
2824            3.5        0.600151        0.640090          4  
366             3.5        0.585570        0.585570          1  
2455            4.0        0.575911        0.626122          6  


## 4. Overview Dashboard

This dashboard provides high-level views of risk scores and ratings over time.
The HTML is saved in the `dashboards/` folder.


In [ ]:
# Time series of average rating and average risk
daily_summary = df.groupby("date", as_index=False).agg(
    avg_rating=("stars_business", "mean"), avg_risk=("risk_score", "mean")
)

fig_overview = go.Figure()
fig_overview.add_trace(
    go.Scatter(
        x=daily_summary["date"],
        y=daily_summary["avg_rating"],
        mode="lines",
        name="Average Rating",
    )
)
fig_overview.add_trace(
    go.Scatter(
        x=daily_summary["date"],
        y=daily_summary["avg_risk"],
        mode="lines",
        name="Average Risk Score",
        yaxis="y2",
    )
)

fig_overview.update_layout(
    title="Overall Trends: Average Rating vs. Average Risk Score",
    xaxis_title="Date",
    yaxis_title="Average Rating",
    yaxis2=dict(title="Average Risk Score", overlaying="y", side="right"),
    legend=dict(orientation="h"),
)

overview_path = "dashboards/dashboard_overview.html"
fig_overview.write_html(overview_path)
print(f"✓ Saved overview dashboard to: {overview_path}")


✓ Saved overview dashboard to: dashboards/dashboard_overview.html


## 5. Alerts Dashboard

This dashboard highlights the top high-risk restaurants based on their
average risk score.


In [ ]:
top_n = 20
top_risk_businesses = business_group.head(top_n).copy()
top_risk_businesses["rank"] = np.arange(1, len(top_risk_businesses) + 1)

fig_alerts = px.bar(
    top_risk_businesses,
    x="avg_risk_score",
    y="name",
    orientation="h",
    hover_data=["latest_rating", "n_reviews"],
    labels={"avg_risk_score": "Average Risk Score", "name": "Business"},
    title=f"Top {top_n} High-Risk Restaurants",
)
fig_alerts.update_layout(yaxis=dict(autorange="reversed"))

alerts_path = "dashboards/dashboard_alerts.html"
fig_alerts.write_html(alerts_path)
print(f"✓ Saved alerts dashboard to: {alerts_path}")


✓ Saved alerts dashboard to: dashboards/dashboard_alerts.html


## 6. LLM Complaint Analysis Dashboard

If LLM complaint analysis is available, we summarize complaint categories
and severity distribution.


In [ ]:
llm_dashboard_path = None

if llm_df is not None and len(llm_df) > 0:
    # Category columns used in the LLM analysis
    category_cols = [
        "food_quality",
        "service_speed",
        "staff_behavior",
        "cleanliness",
        "portion_size",
        "pricing",
        "order_accuracy",
    ]
    available_cols = [c for c in category_cols if c in llm_df.columns]

    # Compute category counts (treat True as 1, others as 0)
    if available_cols:
        category_flags = llm_df[available_cols].applymap(
            lambda v: 1 if v is True else 0
        )
        category_counts = category_flags.sum().sort_values(ascending=False)
        category_df = category_counts.reset_index()
        category_df.columns = ["category", "count"]

        fig_categories = px.bar(
            category_df,
            x="category",
            y="count",
            title="LLM Complaint Category Distribution",
            labels={"category": "Category", "count": "Number of Reviews"},
        )

        # Severity distribution if available
        severity_fig = None
        if "severity" in llm_df.columns:
            sev_counts = llm_df["severity"].value_counts().reset_index()
            sev_counts.columns = ["severity", "count"]
            severity_fig = px.bar(
                sev_counts,
                x="severity",
                y="count",
                title="Complaint Severity Distribution",
                labels={"severity": "Severity", "count": "Number of Reviews"},
            )

        # Combine into a simple HTML report
        from plotly.subplots import make_subplots

        if severity_fig is not None:
            fig_llm = make_subplots(
                rows=1,
                cols=2,
                subplot_titles=("Category Distribution", "Severity Distribution"),
            )
            for trace in fig_categories.data:
                fig_llm.add_trace(trace, row=1, col=1)
            for trace in severity_fig.data:
                fig_llm.add_trace(trace, row=1, col=2)

            fig_llm.update_layout(title_text="LLM Complaint Analysis Summary")
        else:
            fig_llm = fig_categories

        llm_dashboard_path = "dashboards/dashboard_llm_complaints.html"
        fig_llm.write_html(llm_dashboard_path)
        print(f"✓ Saved LLM complaint dashboard to: {llm_dashboard_path}")
    else:
        print(
            "No boolean complaint category columns found in LLM output; skipping category dashboard."
        )
else:
    print("LLM complaint analysis not available; skipping LLM dashboard.")


✓ Saved LLM complaint dashboard to: dashboards/dashboard_llm_complaints.html


/var/folders/j1/ln88kc6d3qn6v_mv1kg4krsc0000gn/T/ipykernel_20918/4176902861.py:13: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



## 7. Generate Textual Dashboard Summary (for Reports)


In [8]:
report_lines = []

report_lines.append("=" * 70)
report_lines.append("DASHBOARD & BUSINESS INSIGHTS SUMMARY (OPTIMIZED MODEL)")
report_lines.append("=" * 70)
report_lines.append("")

# Model performance summary
report_lines.append("Model Performance (Optimized Random Forest):")
report_lines.append(f"  ROC AUC   : {model_summary.get('roc_auc', np.nan):.3f}")
report_lines.append(f"  Accuracy  : {model_summary.get('accuracy', np.nan):.3f}")
report_lines.append(f"  Precision : {model_summary.get('precision', np.nan):.3f}")
report_lines.append(f"  Recall    : {model_summary.get('recall', np.nan):.3f}")
report_lines.append(f"  F1-Score  : {model_summary.get('f1_score', np.nan):.3f}")
report_lines.append(f"  Threshold : {optimal_threshold:.3f}")
report_lines.append("")

# Overall stats
n_businesses = business_group["business_id"].nunique()
n_reviews = len(df)
n_high_risk = (business_group["avg_risk_score"] >= optimal_threshold).sum()

report_lines.append("Overall System Statistics:")
report_lines.append(f"  Businesses monitored : {n_businesses}")
report_lines.append(f"  Reviews analyzed     : {n_reviews}")
report_lines.append(f"  High-risk businesses : {n_high_risk}")
report_lines.append("")

# Top high-risk businesses
top_display = 10
report_lines.append(f"Top {top_display} high-risk businesses:")
for _, row in business_group.head(top_display).iterrows():
    line = (
        f"  - {row['name']} | "
        f"avg_risk={row['avg_risk_score']:.3f}, "
        f"latest_rating={row['latest_rating']:.2f}, "
        f"reviews={int(row['n_reviews'])}"
    )
    report_lines.append(line)

report_lines.append("")

# LLM insights summary (if available)
if llm_df is not None and len(llm_df) > 0:
    report_lines.append("LLM Complaint Analysis (sampled negative reviews):")
    n_llm = len(llm_df)
    report_lines.append(f"  Reviews analyzed by LLM : {n_llm}")

    if "primary_issue" in llm_df.columns:
        top_issues = llm_df["primary_issue"].value_counts().head(5)
        report_lines.append("  Top primary issues:")
        for issue, count in top_issues.items():
            if pd.isna(issue) or not str(issue).strip():
                continue
            pct = (count / n_llm) * 100
            report_lines.append(f"    • {issue} ({int(count)} reviews, {pct:4.1f}%)")
else:
    report_lines.append("LLM complaint analysis not available in this run.")

report_lines.append("")

# Save report
dashboard_report_path = "reports/4_dashboard.txt"
with open(dashboard_report_path, "w", encoding="utf-8") as f:
    f.write("\n".join(report_lines))

print(f"✓ Saved dashboard summary report to: {dashboard_report_path}")


✓ Saved dashboard summary report to: reports/4_dashboard.txt


## 8. Run Summary


In [ ]:
print("\n" + "=" * 70)
print("DASHBOARDS & REPORT GENERATION COMPLETE (OPTIMIZED PIPELINE)")
print("=" * 70)
print("\nGenerated files:")
print("  - dashboards/dashboard_overview.html")
print("  - dashboards/dashboard_alerts.html")
print("  - dashboards/dashboard_llm_complaints.html (if LLM results available)")
print("  - reports/4_dashboard.txt")



DASHBOARDS & REPORT GENERATION COMPLETE (OPTIMIZED PIPELINE)

Generated files:
  - dashboards/dashboard_overview.html
  - dashboards/dashboard_alerts.html
  - dashboards/dashboard_llm_complaints.html (if LLM results available)
  - reports/4_dashboard.txt
